In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

base_points_path = './base-points.csv'
transformed_points_path = './transformed-points.csv'

base_points = np.loadtxt(base_points_path, delimiter=',', skiprows=1)
transformed_points = np.loadtxt(transformed_points_path, delimiter=',', skiprows=1)

neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto')
neighbors.fit(base_points)

distances, indices = neighbors.kneighbors(transformed_points)
base_points[indices]

array([[[-2.50919762,  9.01428613]],

       [[ 4.59212357,  2.75114943]],

       [[-6.36350066, -6.3319098 ]],

       [[-8.83832776,  7.32352292]],

       [[ 2.02230023,  4.16145156]],

       [[-9.58831011,  9.39819704]],

       [[ 6.57475018, -2.86493347]],

       [[-6.36350066, -6.3319098 ]],

       [[-3.76577848,  0.40136042]],

       [[-1.36109963, -4.1754172 ]],

       [[ 5.44489539, -6.02568637]],

       [[-4.15710703, -2.67276313]],

       [[-0.87860032,  5.70351923]],

       [[-4.38130981,  0.85392166]],

       [[ 2.23705789, -7.21012279]],

       [[ 5.44489539, -6.02568637]],

       [[-8.69896814,  8.97771075]],

       [[ 8.78997883,  7.89654701]],

       [[-2.83068543, -7.68261881]],

       [[ 5.4193436 , -0.12408807]],

       [[-6.00652436,  0.28468877]],

       [[-9.31222958,  8.18640804]],

       [[-4.82440037,  3.25044569]],

       [[-3.76577848,  0.40136042]],

       [[ 2.15089704, -6.58951753]],

       [[ 8.78997883,  7.89654701]],

       [[ 8.